In [1]:
import yfinance as yf
sp500_data = yf.download('^GSPC', start='2018-01-01', end='2022-12-31')


[*********************100%%**********************]  1 of 1 completed


In [2]:
print(sp500_data.head())

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2018-01-02  2683.729980  2695.889893  2682.360107  2695.810059  2695.810059   
2018-01-03  2697.850098  2714.370117  2697.770020  2713.060059  2713.060059   
2018-01-04  2719.310059  2729.290039  2719.070068  2723.989990  2723.989990   
2018-01-05  2731.330078  2743.449951  2727.919922  2743.149902  2743.149902   
2018-01-08  2742.669922  2748.510010  2737.600098  2747.709961  2747.709961   

                Volume  
Date                    
2018-01-02  3397430000  
2018-01-03  3544030000  
2018-01-04  3697340000  
2018-01-05  3239280000  
2018-01-08  3246160000  


In [12]:
import psycopg2
from postgres_config import POSTGRES_DB, POSTGRES_HOST, POSTGRES_PASSWORD, POSTGRES_PORT, POSTGRES_USER
conn = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    database=POSTGRES_DB 
)

In [13]:
cur = conn.cursor()
cur.execute("""
    CREATE TABLE sch_nypd_calls_tables.tb_sp_500 (
        Date DATE,
        Open NUMERIC,
        High NUMERIC,
        Low NUMERIC,
        Close NUMERIC,
        Adj_Close NUMERIC,
        Volume BIGINT
    )
""")
conn.commit()


In [14]:
# Reset the index so that 'Date' becomes a regular column
sp500_data_reset = sp500_data.reset_index()

# Now iterate over the DataFrame
for index, row in sp500_data_reset.iterrows():
    cur.execute(
        "INSERT INTO sch_nypd_calls_tables.tb_sp_500 VALUES (%s, %s, %s, %s, %s, %s, %s)",
        (row['Date'], row['Open'], row['High'], row['Low'], row['Close'], row['Adj Close'], row['Volume'])
    )
conn.commit()
